# Exploration notebook

This notebook aims at first collecting a dataset of StackOverflow questions before doing a first exploratory analysis.

## Imports

In [ ]:
!pip uninstall helpers -y

In [ ]:
!pip install git+https://github.com/Xmaster6y/ML-Engineer@package

In [ ]:
!pip install sentence_transformers

In [ ]:
!pip install spacy

In [ ]:
import os
import re
import pickle

import pandas as pd
import numpy as np
import spacy
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px
from numpy.random import default_rng

from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud

from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.manifold import TSNE

from sklearn.preprocessing import MultiLabelBinarizer

from sentence_transformers import SentenceTransformer

In [ ]:
import helpers

In [ ]:
spacy.prefer_gpu()
nlp = spacy.load("en_core_web_sm")

## Data loading

The dataset was generated using the stackoverflow API.

- Dataset `raw_data_50k` :

```sql
SELECT TOP 50000 Title, Body, Tags, Id, Score, ViewCount, AnswerCount
FROM Posts
WHERE (
  PostTypeId = 1 AND AcceptedAnswerId IS NOT NULL
  AND (LEN(Tags) - LEN(REPLACE(Tags, '<','')) >= 4)
  ) AND (
  Score > 20 AND ViewCount > 1000
  )
ORDER BY Score DESC
```

In [ ]:
file_name = "raw_data_50k.csv"
drive_file_id = "1F_-hp4ERdnr7GrCAVSDn_M-NWUOCb_be"
if not os.path.exists(file_name):
    !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=FILEID' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=$drive_file_id" -O $file_name  && rm -rf /tmp/cookies.txt

In [ ]:
df_50k = pd.read_csv(file_name)

In [ ]:
df_50k.info()

In [ ]:
df_50k.head()

## Tags Analysis

In [ ]:
tags_s = df_50k["Tags"].to_list()

In [ ]:
def tag_str_to_list(tag_s:str):
    return tag_s[1:-1].split('><')
tags_l = list(map(tag_str_to_list, tags_s))

In [ ]:
tags_l[:10]

In [ ]:
all_tags = sum(tags_l, start=[])

In [ ]:
occ_tags = {}
for tag in all_tags:
    if tag not in occ_tags:
        occ_tags[tag] = 1
    else:
        occ_tags[tag] += 1
N = len(all_tags)
freq_tags = {t:o/N for t,o in occ_tags.items()}

In [ ]:
sorted_tags = sorted([(f,t) for t,f in freq_tags.items()], reverse=True)
sorted_tags[:10]

In [ ]:
len(sorted_tags)

In [ ]:
sorted_tags[40:50]

In [ ]:
sorted_tags[50:100]

In [ ]:
_, popular_tags = zip(*sorted_tags[:500])

In [ ]:
cleaned_tags_l = [
    [t for t in tag_l if t in popular_tags ] for tag_l in tags_l
]
cleaned_tags_l[:10]

## Tags Encoding

In [ ]:
encoder = MultiLabelBinarizer(classes=popular_tags)
encoded_tags = encoder.fit_transform(cleaned_tags_l)
encoded_tags.shape

In [ ]:
encoded_tags[:10]

## Tags correlation

In [ ]:
corr = np.corrcoef(encoded_tags, rowvar=False)
mask = np.triu(np.ones_like(corr, dtype=bool))

In [ ]:
sns.heatmap(corr, mask=mask)

In [ ]:
np.triu([[1,2,3],[4,5,6],[7,8,9],[10,11,12]], 0)

In [ ]:
len(np.argwhere(corr*(1-mask) > 0.62))

In [ ]:
len(np.argwhere(corr*(1-mask) < -0.1))

In [ ]:
indices = np.argwhere((corr*(1-mask) > 0.62) | (corr*(1-mask) < -0.1))
corr_tags = []
for x, y in indices:
    corr_tags.append((
        corr[x,y],
        popular_tags[x],
        popular_tags[y]
    ))
corr_tags.sort(reverse=True)
corr_tags

In [ ]:
fig, ax = plt.subplots()
n=12
N=1000
cmap = plt.get_cmap('plasma_r', N,)
norm = matplotlib.colors.Normalize(vmin=-1, vmax=1)


for i in range(n//2):
    ax.text(0.2, (n-i+0.4)/(n+1), f'{corr_tags[i][0]:.2f}', color='k', verticalalignment ='center', horizontalalignment ='left')
    ax.text(0.35, (n-i+0.4)/(n+1), f'{corr_tags[i][1:]}', color=cmap(norm(corr_tags[i][0])), verticalalignment ='center', horizontalalignment ='left',
            bbox=dict(facecolor='none', edgecolor=cmap(norm(corr_tags[i][0])), boxstyle='round'))
ax.text(0.35, 0.48, '°°°', color='k', verticalalignment ='center', horizontalalignment ='left', fontsize="xx-large")
for i in range(n//2, n):
    ax.text(0.2, (n-i-0.4)/(n+1), f'{corr_tags[i][0]:.2f}', color='k', verticalalignment ='center', horizontalalignment ='left')
    ax.text(0.35, (n-i-0.4)/(n+1), f'{corr_tags[i][1:]}', color=cmap(norm(corr_tags[i][0])), verticalalignment ='center', horizontalalignment ='left',
            bbox=dict(facecolor='none', edgecolor=cmap(norm(corr_tags[i][0])), boxstyle='round'))
plt.axis('off')


sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
plt.colorbar(sm, location='right')

plt.show()

## Raw title embedding

In [ ]:
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)
embeddings.shape

In [ ]:
titles = df_50k["Title"].to_list()
titles[:10]

In [ ]:
title_embeddings = model.encode(titles)
title_embeddings.shape

## Text processing and analysis

### Processing

In [ ]:
def extract_text_code(body, max_length=1000):
    """Extracts the text and code of an html page
    """
    soup_to_txt = lambda s: s.getText()
    soup = BeautifulSoup(body)
    text_soup_l = soup.find_all(['h1', 'h2', 'h3', 'h4', 'p'])
    text_l = list(map(soup_to_txt, text_soup_l))
    text = '\n'.join(text_l).encode('ascii',errors='ignore').decode('ascii')
    text_cropped=len(text)>max_length

    code_soup_l = soup.find_all('code')
    code_l = list(map(soup_to_txt, code_soup_l))
    code = '\n'.join(code_l).encode('ascii',errors='ignore').decode('ascii')
    code_cropped=len(code)>max_length
    return text[:max_length], text_cropped, code[:max_length], code_cropped

In [ ]:
print(df_50k["Body"][0])

In [ ]:
print(extract_text_code(df_50k["Body"][0])[2])

In [ ]:
text_codes_l = df_50k["Body"].apply(extract_text_code)

In [ ]:
text_l, text_cropped_l, code_l, code_cropped_l = zip(*text_codes_l)

In [ ]:
df_final = df_50k
df_final = df_final.drop(columns=["Body", "Id", "AcceptedAnswerId"])
df_final["BodyText"]= text_l
df_final["BodyCode"]= code_l
df_final["BodyTextCropped"]= text_cropped_l
df_final["BodyCodeCropped"]= code_cropped_l

In [ ]:
df_final.head()

In [ ]:
df_final.to_csv("df_cleaned.csv", index=False)

### Vocabulary analysis

In [ ]:
N = 2000
rng = default_rng(seed=6)
numbers = rng.choice(len(df_final), size=N, replace=False)

In [ ]:
def lemmatize(text):
   doc = nlp(text)
   tokens = [token.lemma_ for token in doc if not (token.is_stop or token.is_punct)]
   return ' '.join(tokens)

In [ ]:
bow = CountVectorizer(preprocessor = lemmatize)

In [ ]:
bow_title = bow.fit_transform(df_final.loc[numbers,"Title"].to_list())
bow_title_voc = bow.vocabulary_
bow_body_text = bow.fit_transform(df_final.loc[numbers,"BodyText"].to_list())
bow_body_text_voc = bow.vocabulary_
bow_body_code = bow.fit_transform(df_final.loc[numbers,"BodyCode"].to_list())
bow_body_code_voc = bow.vocabulary_

In [ ]:
print(bow_title.shape)
print(bow_body_text.shape)
print(bow_body_code.shape)

In [ ]:
wordcloud = WordCloud(width= 1000, height = 600, max_words=100,
                      random_state=1, background_color='white', colormap='viridis_r',
                      collocations=False).generate_from_frequencies(bow_title_voc)
plt.figure()
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

In [ ]:
wordcloud = WordCloud(width= 1000, height = 600, max_words=100,
                      random_state=1, background_color='white', colormap='viridis_r',
                      collocations=False).generate_from_frequencies(bow_body_text_voc)
plt.figure()
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

In [ ]:
wordcloud = WordCloud(width= 1000, height = 600, max_words=100,
                      random_state=1, background_color='white', colormap='viridis_r',
                      collocations=False).generate_from_frequencies(bow_body_code_voc)
plt.figure()
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

## Dimension reduction

In [ ]:
title_embeddings_sub = title_embeddings[numbers]

text_embeddings_sub = model.encode(df_final.loc[numbers,"BodyText"].to_list())
code_embeddings_sub = model.encode(df_final.loc[numbers,"BodyCode"].to_list())


In [ ]:
titlecode_embeddings_sub = model.encode((df_final.loc[numbers,"Title"]+" "+df_final.loc[numbers,"BodyCode"]).to_list())
all_embeddings_sub = model.encode((df_final.loc[numbers,"Title"]+" "+df_final.loc[numbers,"BodyText"]+" "+df_final.loc[numbers,"BodyCode"]).to_list())

In [ ]:
def to_name(encoded_tag):
    for i in range(len(encoded_tag)-1,-1,-1):
        if encoded_tag[i]:
            return popular_tags[i]
    return "none"

pseudo_labels = list(map(to_name, encoded_tags[numbers,:10]))

### PCA

In [ ]:
n_components=title_embeddings_sub.shape[1]
title_pca = PCA(n_components=n_components)
text_pca = PCA(n_components=n_components)
code_pca = PCA(n_components=n_components)
titlecode_pca = PCA(n_components=n_components)
all_pca = PCA(n_components=n_components)

In [ ]:
X_title_proj = title_pca.fit_transform(title_embeddings_sub)
X_text_proj = text_pca.fit_transform(text_embeddings_sub)
X_code_proj = code_pca.fit_transform(code_embeddings_sub)
X_titlecode_proj = titlecode_pca.fit_transform(titlecode_embeddings_sub)
X_all_proj = all_pca.fit_transform(all_embeddings_sub)

In [ ]:
scree = (title_pca.explained_variance_ratio_*100)
scree_cum = scree.cumsum()
x_list = range(1, n_components+1)
plt.bar(x_list, scree)
plt.plot(x_list, scree_cum,c="red",marker='o')
plt.xlabel("Inertia rank")
plt.ylabel("Inertia percentage")
plt.title("Eigen value cumulative graph")
plt.show(block=False)

In [ ]:
scree = (text_pca.explained_variance_ratio_*100)
scree_cum = scree.cumsum()
x_list = range(1, n_components+1)
plt.bar(x_list, scree)
plt.plot(x_list, scree_cum,c="red",marker='o')
plt.xlabel("Inertia rank")
plt.ylabel("Inertia percentage")
plt.title("Eigen value cumulative graph")
plt.show(block=False)

In [ ]:
scree = (code_pca.explained_variance_ratio_*100)
scree_cum = scree.cumsum()
x_list = range(1, n_components+1)
plt.bar(x_list, scree)
plt.plot(x_list, scree_cum,c="red",marker='o')
plt.xlabel("Inertia rank")
plt.ylabel("Inertia percentage")
plt.title("Eigen value cumulative graph")
plt.show(block=False)

In [ ]:
scree = (titlecode_pca.explained_variance_ratio_*100)
scree_cum = scree.cumsum()
x_list = range(1, n_components+1)
plt.bar(x_list, scree)
plt.plot(x_list, scree_cum,c="red",marker='o')
plt.xlabel("Inertia rank")
plt.ylabel("Inertia percentage")
plt.title("Eigen value cumulative graph")
plt.show(block=False)

In [ ]:
scree = (all_pca.explained_variance_ratio_*100)
scree_cum = scree.cumsum()
x_list = range(1, n_components+1)
plt.bar(x_list, scree)
plt.plot(x_list, scree_cum,c="red",marker='o')
plt.xlabel("Inertia rank")
plt.ylabel("Inertia percentage")
plt.title("Eigen value cumulative graph")
plt.show(block=False)

In [ ]:
sns.barplot(x=np.arange(n_components),y=code_pca.components_[0,:])
ax=plt.gca()
ax.set_xticks(np.arange(n_components,step=50))
plt.xlabel("feature")
plt.ylabel("value")
plt.title("Component 0 for code embedding")

In [ ]:
x_y = (0,1)
helpers.plot.pca.display_factorial_planes(X_title_proj, x_y, clusters=pseudo_labels)

In [ ]:
x_y = (2,3)
helpers.plot.pca.display_factorial_planes(X_title_proj, x_y, clusters=pseudo_labels)

In [ ]:
x_y = (1,2)
helpers.plot.pca.display_factorial_planes(X_title_proj, x_y, clusters=pseudo_labels)

In [ ]:
x_y = (0,1)
helpers.plot.pca.display_factorial_planes(X_text_proj, x_y, clusters=pseudo_labels)

In [ ]:
x_y = (0,1)
helpers.plot.pca.display_factorial_planes(X_code_proj, x_y, clusters=pseudo_labels)

In [ ]:
x_y = (2,3)
helpers.plot.pca.display_factorial_planes(X_code_proj, x_y, clusters=pseudo_labels)

In [ ]:
x_y = (1,2)
helpers.plot.pca.display_factorial_planes(X_code_proj, x_y, clusters=pseudo_labels)

In [ ]:
x_y = (0,1)
helpers.plot.pca.display_factorial_planes(X_all_proj, x_y, clusters=pseudo_labels)

In [ ]:
x_y = (0,1)
helpers.plot.pca.display_factorial_planes(X_titlecode_proj, x_y, clusters=pseudo_labels)

In [ ]:
x_y = (2,3)
helpers.plot.pca.display_factorial_planes(X_titlecode_proj, x_y, clusters=pseudo_labels)

### t-SNE

In [ ]:
tsne = TSNE(n_components=2, verbose=1, perplexity=40.0, random_state=42)
z = tsne.fit_transform(title_embeddings_sub)

In [ ]:
df = pd.DataFrame()
df["hue"] = pseudo_labels
df["comp-1"] = z[:,0]
df["comp-2"] = z[:,1]

sns.scatterplot(x="comp-1", y="comp-2", hue="hue",
                palette=sns.color_palette("hls", 11),
                data=df).set(title="T-SNE projection")

In [ ]:
tsne = TSNE(n_components=2, verbose=1, perplexity=40.0, random_state=42)
z = tsne.fit_transform(code_embeddings_sub)

In [ ]:
df = pd.DataFrame()
df["hue"] = pseudo_labels
df["comp-1"] = z[:,0]
df["comp-2"] = z[:,1]

sns.scatterplot(x="comp-1", y="comp-2", hue="hue",
                palette=sns.color_palette("hls", 11),
                data=df).set(title="T-SNE projection")

In [ ]:
tsne = TSNE(n_components=2, verbose=1, perplexity=40.0, random_state=42)
z = tsne.fit_transform(all_embeddings_sub)

In [ ]:
df = pd.DataFrame()
df["hue"] = pseudo_labels
df["comp-1"] = z[:,0]
df["comp-2"] = z[:,1]

sns.scatterplot(x="comp-1", y="comp-2", hue="hue",
                palette=sns.color_palette("hls", 11),
                data=df).set(title="T-SNE projection")

In [ ]:
perplexity = np.linspace(5,50,10, dtype=int)
divergence = []

for i in perplexity:
    model = TSNE(n_components=2, init="pca", perplexity=i, verbose=1)
    reduced = model.fit_transform(title_embeddings_sub)
    divergence.append(model.kl_divergence_)
fig = px.line(x=perplexity, y=divergence, markers=True)
fig.update_layout(xaxis_title="Perplexity Values", yaxis_title="Divergence")
fig.update_traces(line_color="red", line_width=1)
fig.show()

In [ ]:
plt.plot(perplexity, 2.*np.array(divergence)+np.log(N)*perplexity/N,c="red",marker='o')
ymin, ymax = plt.ylim()
plt.vlines(40, ymin=ymin, ymax=ymax)
plt.ylim(ymin, ymax )
plt.xlabel("Perplexity")
plt.ylabel("S criterion")
plt.show(block=False)

In [ ]:
perplexity = np.linspace(5,50,10, dtype=int)
divergence = []

for i in perplexity:
    model = TSNE(n_components=2, init="pca", perplexity=i, verbose=1)
    reduced = model.fit_transform(code_embeddings_sub)
    divergence.append(model.kl_divergence_)
fig = px.line(x=perplexity, y=divergence, markers=True)
fig.update_layout(xaxis_title="Perplexity Values", yaxis_title="Divergence")
fig.update_traces(line_color="red", line_width=1)
fig.show()

In [ ]:
plt.plot(perplexity, 2.*np.array(divergence)+np.log(N)*perplexity/N,c="red",marker='o')
ymin, ymax = plt.ylim()
plt.vlines(40, ymin=ymin, ymax=ymax)
plt.ylim(ymin, ymax )
plt.xlabel("Perplexity")
plt.ylabel("S criterion")
plt.show(block=False)

In [ ]:
tsne = TSNE(n_components=2, verbose=1, perplexity=5, random_state=42)
z = tsne.fit_transform(code_embeddings_sub)

In [ ]:
df = pd.DataFrame()
df["hue"] = pseudo_labels
df["comp-1"] = z[:,0]
df["comp-2"] = z[:,1]

sns.scatterplot(x="comp-1", y="comp-2", hue="hue",
                palette=sns.color_palette("hls", 11),
                data=df).set(title="T-SNE projection")

In [ ]:
tsne = TSNE(n_components=2, verbose=1, perplexity=1000, random_state=42)
z = tsne.fit_transform(code_embeddings_sub)

In [ ]:
df = pd.DataFrame()
df["hue"] = pseudo_labels
df["comp-1"] = z[:,0]
df["comp-2"] = z[:,1]

sns.scatterplot(x="comp-1", y="comp-2", hue="hue",
                palette=sns.color_palette("hls", 11),
                data=df).set(title="T-SNE projection")

## Export

In [ ]:
export_name = "title_embedding_50k_500t.pkl"

In [ ]:
with open(export_name, "wb") as f:
    pickle.dump((title_embeddings, encoded_tags, popular_tags), f)